In [6]:
import pandas as pd


abbreviations_df = pd.read_csv('teams.csv')
mapping_dict = dict(zip(abbreviations_df['Long'], abbreviations_df['Abbrv']))


def update_and_overwrite_file(file_path, mapping_dict):
    df = pd.read_csv(file_path)
    df['AwayTeam'] = df['AwayTeam'].map(mapping_dict)
    df['HomeTeam'] = df['HomeTeam'].map(mapping_dict)
    df.to_csv(file_path, index=False)

file_paths = ['23_24_sched.csv']


for file_path in file_paths:
    update_and_overwrite_file(file_path, mapping_dict)

In [8]:
import pandas as pd
import re

file_paths = ['23_24_sched.csv']
df = pd.read_csv(file_path)
pattern = r'(\d{4}-\d{2}-\d{2}) - ([\w\s]+), ([\w\s]+)'
df['Game'] = df['Date'] + ' - ' + df['AwayTeam'] + ', ' + df['HomeTeam']
df.drop('Unnamed: 4', axis=1, inplace=True)
df.to_csv(file_path, index=False)  




,Date,EST,AwayTeam,HomeTeam,Game
0,2023-10-10,5:30 PM,NSH,TB,"2023-10-10 - NSH, TB"
1,2023-10-10,8:00 PM,CHI,PIT,"2023-10-10 - CHI, PIT"
2,2023-10-10,10:30 PM,SEA,VGK,"2023-10-10 - SEA, VGK"
3,2023-10-11,7:00 PM,OTT,CAR,"2023-10-11 - OTT, CAR"
4,2023-10-11,7:00 PM,MTL,TOR,"2023-10-11 - MTL, TOR"


In [13]:
import pandas as pd


df = pd.read_csv('23_24_sched.csv')
df1 = df[['Date','AwayTeam','HomeTeam','Game']]
df1.head()




,Date,AwayTeam,HomeTeam,Game
0,2023-10-10,NSH,TB,"2023-10-10 - NSH, TB"
1,2023-10-10,CHI,PIT,"2023-10-10 - CHI, PIT"
2,2023-10-10,SEA,VGK,"2023-10-10 - SEA, VGK"
3,2023-10-11,OTT,CAR,"2023-10-11 - OTT, CAR"
4,2023-10-11,MTL,TOR,"2023-10-11 - MTL, TOR"


In [16]:
games_df = df1

def add_home_team_stats(row):
    team_name = row['HomeTeam']
    file_path = f'23_24_average_data/{team_name}.csv'

    # Load the home team stats
    team_stats_df = pd.read_csv(file_path)

    # Find the row with the matching game ID
    game_row = team_stats_df[team_stats_df['Date'] == row['Date']]

    if not game_row.empty:
        # Add prefix to column names, except for 'Game'
        team_stats = game_row.add_prefix('h_').iloc[0]
        team_stats.rename({'h_Game': 'Game'}, inplace=True)

        # Update the game row in games_df with home team stats
        for col in team_stats.index:
            row[col] = team_stats[col]

    return row

# Iterate over each game in the DataFrame and add home team stats
updated_rows = []
for index, row in games_df.iterrows():
    row = add_home_team_stats(row)  # Add home team stats
    updated_rows.append(row)
hgames_df = pd.DataFrame(updated_rows)

df2 = hgames_df.dropna()
df2.head()

,Date,AwayTeam,HomeTeam,Game,h_Date,h_days_of_rest,h_CA,h_CF,h_FA,h_FF,...,h_MDGF,h_MDSA,h_MDSF,h_PDO,h_SA,h_SCA,h_SCF,h_SF,h_xGA,h_xGF
146,2023-11-02,FLA,DET,"2023-11-02 - FLA 2, DET 0",2023-11-02,2.0,64.6,57.2,48.2,42.8,...,1.2,7.8,8.7,1.0334,31.4,32.2,27.9,31.0,3.657,3.192
153,2023-11-02,NSH,SEA,"2023-11-02 - NSH 2, SEA 4",2023-11-02,2.0,65.1,64.1,47.9,46.1,...,0.7,7.4,9.2,0.9648,32.5,27.2,29.0,32.0,3.230,3.037
154,2023-11-02,WPG,VGK,"2023-11-02 - WPG 2, VGK 5",2023-11-02,2.0,62.6,58.8,43.9,39.9,...,1.2,7.0,7.9,1.0492,29.9,29.6,30.0,28.8,3.005,3.046
156,2023-11-03,PHI,BUF,"2023-11-03 - PHI 5, BUF 1",2023-11-03,1.0,60.8,55.2,44.9,39.7,...,1.7,7.8,8.5,1.0397,31.3,28.8,25.3,28.5,3.423,2.743
159,2023-11-04,WPG,ARI,"2023-11-04 - WPG 5, ARI 3",2023-11-04,1.0,59.4,57.2,44.0,42.1,...,0.7,10.0,8.9,1.0173,31.5,29.6,30.4,29.7,2.864,3.300


In [19]:
games_df = df1

def add_away_team_stats(row):
    team_name = row['AwayTeam']
    file_path = f'23_24_average_data/{team_name}.csv'

    # Load the home team stats
    team_stats_df = pd.read_csv(file_path)

    # Find the row with the matching game ID
    game_row = team_stats_df[team_stats_df['Date'] == row['Date']]

    if not game_row.empty:
        # Add prefix to column names, except for 'Game'
        team_stats = game_row.add_prefix('a_').iloc[0]
        team_stats.rename({'a_Game': 'Game'}, inplace=True)

        # Update the game row in games_df with home team stats
        for col in team_stats.index:
            row[col] = team_stats[col]

    return row

# Iterate over each game in the DataFrame and add home team stats
updated_rows = []
for index, row in games_df.iterrows():
    row = add_away_team_stats(row)  # Add home team stats
    updated_rows.append(row)
agames_df = pd.DataFrame(updated_rows)


In [28]:
import pandas as pd
common_columns = ['Game', 'Date', 'AwayTeam', 'HomeTeam']

merged_df = pd.merge(agames_df, hgames_df, on=common_columns, how='outer')
df2 = merged_df.dropna()
pattern = r'(\d{4}-\d{2}-\d{2}) - ([\w\s]+) (\d+), ([\w\s]+) (\d+)'
df2[['Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore']] = df2['Game'].str.extract(pattern)
df2.loc[df2['HomeScore'] > df2['AwayScore'], 'HomeResult'] = 'Won'
df2.loc[df2['HomeScore'] < df2['AwayScore'], 'HomeResult'] = 'Lost'
cols_to_drop = ['a_Date','h_Date']
df2.drop(columns=cols_to_drop, inplace=True)

df2.to_csv('testing_set.csv', index=False)

C:\Users\luken\AppData\Local\Temp\ipykernel_10008\3057616961.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore']] = df2['Game'].str.extract(pattern)
C:\Users\luken\AppData\Local\Temp\ipykernel_10008\3057616961.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore']] = df2['Game'].str.extract(pattern)
C:\Users\luken\AppData\Local\Temp\ipykernel_10008\3057616961.py:7: SettingWithCopyWarnin